<a href="https://colab.research.google.com/github/DinithYM/ML-CW/blob/main/CM2604_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Telco Customer Churn Prediction

End-to-end churn prediction using Decision Tree and Neural Network on the Telco Customer Churn dataset.
